# Ensemble Learning

## Initial Imports

In [190]:
import warnings
warnings.filterwarnings('ignore')

In [191]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [192]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [193]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [194]:
#encode dummy varibles 
df_dummy = pd.get_dummies(df, columns=["home_ownership", "verification_status", 
                                       "initial_list_status","application_type"])

#initial data clean up
df_clean1 = df_dummy.drop(["issue_d","next_pymnt_d","pymnt_plan",
                           "hardship_flag","debt_settlement_flag",
                           "initial_list_status_f", "application_type_Individual"], axis=1)

## Split the Data into Training and Testing

In [195]:
# Create our features
X = df_clean1.drop(columns="loan_status")

# Create our target
y = df_clean1["loan_status"]

In [196]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_il_high_credit_limit,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,5.572240e+04,0.009285,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001,0.876121,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,5.095845e+04,0.095914,0.499311,0.308793,0.479314,0.499520,0.483865,0.355104,0.329446,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.270000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,2.288000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,4.200000e+04,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,7.249900e+04,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.426964e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [197]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [198]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
                                            
X_train

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_il_high_credit_limit,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,application_type_Joint App
39692,10000.0,0.1131,328.86,49000.0,24.54,0.0,0.0,10.0,1.0,6580.0,...,54197.0,0,1,0,0,0,1,0,1,0
37830,21000.0,0.1072,684.74,57400.0,34.52,0.0,0.0,12.0,0.0,18688.0,...,30000.0,0,0,0,1,1,0,0,1,0
64531,35000.0,0.0881,1109.90,180000.0,3.01,0.0,2.0,7.0,0.0,836.0,...,26177.0,0,0,1,0,1,0,0,1,0
28730,20000.0,0.0819,628.49,80000.0,26.51,0.0,0.0,15.0,0.0,2615.0,...,111641.0,0,0,1,0,0,1,0,1,0
35102,12600.0,0.1356,427.96,71000.0,22.38,1.0,2.0,7.0,0.0,9073.0,...,50182.0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18872,35000.0,0.0881,1109.90,400000.0,13.79,0.0,1.0,17.0,1.0,7810.0,...,244399.0,0,1,0,0,0,1,0,1,0
6415,16800.0,0.0881,532.76,31000.0,20.48,0.0,0.0,3.0,0.0,1867.0,...,16900.0,0,0,1,0,0,0,1,1,0
12632,25000.0,0.1102,818.71,90000.0,15.27,1.0,0.0,18.0,0.0,26106.0,...,33181.0,0,1,0,0,1,0,0,1,0
24022,14000.0,0.1072,456.49,180000.0,12.35,2.0,0.0,10.0,0.0,3601.0,...,63655.0,0,1,0,0,1,0,0,1,0


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [199]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
Standard_Scaler = StandardScaler()

In [200]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scale = Standard_Scaler.fit(X_train)

In [201]:
# Scale the training and testing data
X_train_scale = X_scale.transform(X_train)
X_test_scale = X_scale.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [202]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
balanced_rfc = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
balanced_rfc.fit(X_train_scale, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [203]:
# Calculated the balanced accuracy score
y_pred_rfc = balanced_rfc.predict(X_test_scale)
balanced_accuracy_score(y_test, y_pred_rfc)

0.7381179722091286

In [204]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_rfc)

array([[   53,    34],
       [ 2276, 14842]], dtype=int64)

In [205]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_rfc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.61      0.87      0.04      0.73      0.51        87
   low_risk       1.00      0.87      0.61      0.93      0.73      0.54     17118

avg / total       0.99      0.87      0.61      0.92      0.73      0.54     17205



In [206]:
# List the features sorted in descending order by feature importance
importances = balanced_rfc.feature_importances_
importances_sorted = sorted(zip(balanced_rfc.feature_importances_, X.columns), reverse=True)
display(importances_sorted)

[(0.06641688833399047, 'total_rec_prncp'),
 (0.06308871156823909, 'last_pymnt_amnt'),
 (0.06207524922360791, 'total_rec_int'),
 (0.05973740522398676, 'total_pymnt_inv'),
 (0.057419826226152645, 'total_pymnt'),
 (0.030385892995704336, 'int_rate'),
 (0.01916785354448492, 'annual_inc'),
 (0.019106715023097187, 'dti'),
 (0.01796512961425096, 'max_bal_bc'),
 (0.017736826136837698, 'mths_since_recent_inq'),
 (0.017444024010477146, 'bc_util'),
 (0.0168458030288213, 'il_util'),
 (0.016401009618315338, 'revol_bal'),
 (0.016214667820251563, 'out_prncp'),
 (0.015898053502618102, 'avg_cur_bal'),
 (0.015827239839913215, 'installment'),
 (0.015701331500487803, 'mo_sin_old_rev_tl_op'),
 (0.015698498104685894, 'tot_cur_bal'),
 (0.015376852612649783, 'out_prncp_inv'),
 (0.015084257280383255, 'total_bc_limit'),
 (0.014963603278125279, 'mo_sin_old_il_acct'),
 (0.01483257779568544, 'tot_hi_cred_lim'),
 (0.014458601361694507, 'all_util'),
 (0.014360270290469603, 'total_rev_hi_lim'),
 (0.013935806152764988,

### Easy Ensemble Classifier

In [207]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
easy_ec = EasyEnsembleClassifier(n_estimators=10, random_state=1)
easy_ec.fit(X_train_scale, y_train)

EasyEnsembleClassifier(random_state=1)

In [208]:
# Calculated the balanced accuracy score
y_pred_eec = easy_ec.predict(X_test_scale)
balanced_accuracy_score(y_test, y_pred_eec)

0.7454292920136497

In [209]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_eec)

array([[   60,    27],
       [ 3403, 13715]], dtype=int64)

In [210]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_eec))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.69      0.80      0.03      0.74      0.55        87
   low_risk       1.00      0.80      0.69      0.89      0.74      0.56     17118

avg / total       0.99      0.80      0.69      0.88      0.74      0.56     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Easy Ensemble Classifier has the 0.7454292920136497 higher than Balanced Random Forest Classifier

2. Which model had the best recall score?

   Balanced Random Forest Classifier has the score of 0.87 total which is 0.07 higher than Easy Ensemble Classifier

3. Which model had the best geometric mean score?

    Easy Ensemble Classifier has the score of 0.74 which is 0.01 higher than the other 

4. What are the top three features?

    'total_rec_prncp'
    'last_pymnt_amnt'
    'total_rec_int'